In [1]:
# paquetes necesarios

import csv # para escribir .csv
import pandas as pd # para manejo de estructuras pandas DataFrame
import os  # para leer .csv
import time # se importa sleep para pausar el código en tiempo controlado
from time import sleep
# Paquetes para utilizar navegador
from msedge.selenium_tools import Edge, EdgeOptions  
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common import exceptions

**Instructivo para utilizar Selenium:** en el navegador EDGE busque la versión: En "Ayuda y comentarios", luego en "Acerca de Microsoft EDGE" aparece la versión.

En la página : https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/#downloads descarge la versión que sea acorde a la versión de EDGE.

En el .rar de descarga extraer "msedgedriver.exe" y ponerlo en una carpeta de acceso de variables del entorno del sistema. Para esto  buscar "variables" en el buscador de inicio de windows e ingresar a "editar las variables del entorno del sistema"

Luego, ingresar a "variables de entorno" , esto abrirá una ventana nueva, en la cual hay 2 secciones: "Variables de usuario" y "variables del sistema". En "variables del sistema" buscar "Path" y hacer doble click. Esto abrirá una lista con todas las carpetas que se pueden utilizar. También si es necesario se puede agregar una neuva carpeta. En estas carpetas se puede copiar el "msedgedriver.exe" y listo. 

Un tutorial para esto: https://www.youtube.com/watch?v=9XAH_TvxwLg :How To Install SELENIUM for EDGE Chromium | PYTHON

Este paso es para que se puedan realizar las acciones en el navegador desde Pytho.

**Definición de funciones utilizadas** : son la opreación del navegador a través de python

In [2]:
##########################################################################################################################################################
#################### ESTE CÓDIGO DEPENDE DIRECTAMENTE DEL IDIOMA EN QUE ESTÉ LA CUENTA DE TWITTER ASOCIADA                  ##############################
#################### ESTA REALIZADO EN ÍNGLES Y NO PUEDE EJECUTRASE EN OTRO IDIOMA SIN CAMBIAR TODAS LAS CELDAS DE BÚSQUEDA ##############################
#################### LOS NOMBRES DE LAS BUSQUEDAS EN EL HTML ESTAN EN INGLES Y CAMBIAN SEGUN EL IDIOMA DE LA CUENTA         ##############################

# crea la instancia para abrir el webbrowser MS EDGE
def create_webdriver_instance():
    options = EdgeOptions()
    options.use_chromium = True
    driver = Edge(options=options)
    return driver

# Ingresa a la pagina de twitter, ingresa credenciales, si falla devuelve error y detiene la operación. Entra y luego va a la pestaña home
def login_to_twitter(username, password, driver):
    url = 'https://twitter.com/login'
    try:
        driver.get(url)
        xpath_username = '//input[@name="session[username_or_email]"]'
        WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.XPATH, xpath_username)))
        uid_input = driver.find_element_by_xpath(xpath_username)
        uid_input.send_keys(username)
    except exceptions.TimeoutException:
        print("Timeout while waiting for Login screen")
        return False

    pwd_input = driver.find_element_by_xpath('//input[@name="session[password]"]')
    pwd_input.send_keys(password)
    try:
        pwd_input.send_keys(Keys.RETURN)
        url = "https://twitter.com/home"
        WebDriverWait(driver, 10).until(expected_conditions.url_to_be(url))
    except exceptions.TimeoutException:
        print("Timeout while waiting for home screen")
    return True

#ingresa el término de busqueda a la casilla de búsqueda de twitter

## !!!!!!!!!! si twitter esta en ingles el área de búsqueda tiene un valor de "Search query"
## pero en español tiene un valor de "Búsqueda"
def find_search_input_and_enter_criteria(search_term, driver):
    xpath_search = '//input[@aria-label="Search query"]'  # Inglés: Search query, Españo : Búsqueda
    search_input = driver.find_element_by_xpath(xpath_search)
    search_input.send_keys(search_term)
    search_input.send_keys(Keys.RETURN)
    return True


# cliquea la opcion "tab_name", se utiliza para ir a la opción "Latest", de los tweets 
def change_page_sort(tab_name, driver):
    """Options for this program are `Latest` and `Top`"""
    tab = driver.find_element_by_link_text(tab_name)
    tab.click()
    xpath_tab_state = f'//a[contains(text(),\"{tab_name}\") and @aria-selected=\"true\"]'

# genera id de tweet para identificalo y ver que no se repita  
def generate_tweet_id(tweet):
    return ''.join(tweet)

# operación de scrolling retiene la última posición para comparar si se movió o no y así detener la operación
# tiene los parámetros de tiempo que espera para realizar el scroll ( para que cargue la página)
# max_attempts es el número de intentos de scrolling para detener la operación
# cuando exede el número de intentos se detiene, esto para cuando se llegue al final de la cadena de búsqueda
#  num_seconds_to_load es el tiempo que espera para realizar el scroll, entonces si carga rápido la pagina se puede reducir, si tarda en cargar se puede aumentar
def scroll_down_page(driver, last_position, num_seconds_to_load=2, scroll_attempt=0, max_attempts=5):
    """The function will try to scroll down the page and will check the current
    and last positions as an indicator. If the current and last positions are the same after `max_attempts`
    the assumption is that the end of the scroll region has been reached and the `end_of_scroll_region`
    flag will be returned as `True`"""
    end_of_scroll_region = False
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # (0 , document.body.scrollHeight) son las coordenadas para mover la barra de scroll
    sleep(num_seconds_to_load)
    curr_position = driver.execute_script("return window.pageYOffset;")
    if curr_position == last_position:
        if scroll_attempt < max_attempts:
            end_of_scroll_region = True
        else:
            scroll_down_page(last_position, curr_position, scroll_attempt + 1) # se compara la posición para ver si se está moviendo o no, de forma de determinar si se llega al final
    last_position = curr_position
    return last_position, end_of_scroll_region

#guarda el tweet
def save_tweet_data_to_csv(records, filepath, mode='a+'):
    header = ['User', 'Handle', 'PostDate', 'TweetText', 'ReplyCount', 'RetweetCount', 'LikeCount']
    with open(filepath, mode=mode, newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        if mode == 'w':
            writer.writerow(header)
        if records:
            writer.writerow(records)
# extrae todos los tweets que hay cargados en la página, para despues realizar el scroll e iterar nuevamente
# twitter carga dinámicamente los tweets en el código html, entonces para que sean cargados a la página se debe
# realizar el scroll, de otra forma no se cargan.
def collect_all_tweets_from_current_view(driver, lookback_limit=25):
    """The page is continously loaded, so as you scroll down the number of tweets returned by this function will
     continue to grow. To limit the risk of 're-processing' the same tweet over and over again, you can set the
     `lookback_limit` to only process the last `x` number of tweets extracted from the page in each iteration.
     You may need to play around with this number to get something that works for you. I've set the default
     based on my computer settings and internet speed, etc..."""
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
    if len(page_cards) <= lookback_limit:
        return page_cards
    else:
        return page_cards[-lookback_limit:]

# extrae la información del tweet
def extract_data_from_current_tweet_card(card):
    try:
        user = card.find_element_by_xpath('.//span').text
    except exceptions.NoSuchElementException:
        user = ""
    except exceptions.StaleElementReferenceException:
        return
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except exceptions.NoSuchElementException:
        handle = ""
    try:
        """
        If there is no post date here, there it is usually sponsored content, or some
        other form of content where post dates do not apply. You can set a default value
        for the postdate on Exception if you which to keep this record. By default I am
        excluding these.
        """
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except exceptions.NoSuchElementException:
        return
    try:
        _comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    except exceptions.NoSuchElementException:
        _comment = ""
    try:
        _responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    except exceptions.NoSuchElementException:
        _responding = ""
    tweet_text = _comment + _responding
    try:
        reply_count = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    except exceptions.NoSuchElementException:
        reply_count = ""
    try:
        retweet_count = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    except exceptions.NoSuchElementException:
        retweet_count = ""
    try:
        like_count = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    except exceptions.NoSuchElementException:
        like_count = ""

    tweet = (user, handle, postdate, tweet_text, reply_count, retweet_count, like_count) # agregar reply_to
    return tweet

# cuerpo principal del programa
def main(username, password, search_term, filepath, page_sort='Latest'):
    save_tweet_data_to_csv(None, filepath, 'w')  # create file for saving records
    last_position = None
    end_of_scroll_region = False
    unique_tweets = set()

    driver = create_webdriver_instance()
    time.sleep(2) #####################
    logged_in = login_to_twitter(username, password, driver)
    time.sleep(2) #######################
    if not logged_in:
        return
    for busqueda in search_term:
        change_page_sort('Home', driver)
        time.sleep(0.5)
        search_found = find_search_input_and_enter_criteria(busqueda, driver)
        if not search_found:
            return
        time.sleep(0.5)
        change_page_sort(page_sort, driver)

        while not end_of_scroll_region:
            cards = collect_all_tweets_from_current_view(driver)
            for card in cards:
                try:
                    tweet = extract_data_from_current_tweet_card(card)
                except exceptions.StaleElementReferenceException:
                    continue
                if not tweet:
                    continue
                tweet_id = generate_tweet_id(tweet)
                if tweet_id not in unique_tweets:
                    unique_tweets.add(tweet_id)
                    save_tweet_data_to_csv(tweet, filepath)
            last_position, end_of_scroll_region = scroll_down_page(driver, last_position)
    driver.quit()


**1)Creación de intancia del browser**: Se ingresan las credenciales y el url (página donde ingresar). 1) se abre el browser. 2) se ingresan las credenciales

In [ ]:

url = 'https://twitter.com/login'

username = "usuario123"            # nombre de usuario o correo
password = "clave123"                 # contraseña
filepath = '#EstásEstamos.csv'        # nombre de archivo de extracción (el nombre del .csv que se guarda)
search_term = '#EstásEstamos'         # termino de busqueda ( se ingresa textual sin " " en la barra de búsqueda)
page_sort = "Latest"                  


################################### sección necesaria si se ejecuta como main()
save_tweet_data_to_csv(None, filepath, 'w')  # create file for saving records
last_position = None
end_of_scroll_region = False
unique_tweets = set()
###################################

############# inicia browser
#create_webdriver_instance()
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)
time.sleep(2)
############ ingresa al url
driver.get(url)
######################### ingresa usuario en la pagina
time.sleep(1)
xpath_username = '//input[@name="session[username_or_email]"]'
uid_input = driver.find_element_by_xpath(xpath_username)
uid_input.send_keys(username)
###################### ingresa contraseña ## se detecta actividad inusual
time.sleep(1)
pwd_input = driver.find_element_by_xpath('//input[@name="session[password]"]')
pwd_input.send_keys(password)
pwd_input.send_keys(Keys.RETURN)

**2)Ingreso en barra de búsqueda**: se ingresa término de búsqueda en barra de búsqueda y se presiona enter. Luego se cambia a la ventana "Latest"

In [ ]:
############################ Ingreso de término de búsqueda en barra de búsqueda
xpath_search = '//input[@aria-label="Search query"]' 
search_input = driver.find_element_by_xpath(xpath_search)
search_input.send_keys(search_term[0])
search_input.send_keys(Keys.RETURN)
time.sleep(1) # se espera que cargue 1 seg
change_page_sort(page_sort, driver) # se cambia a la pestaña page_sort, en este caso "Latest"

**3)Cuerpo prinicpal de copia de Tweets:** Esta rutina extrae los tweets presentes en el código HTML, los cuales corresponen a los que se muestra en pantalla y un poco más.

Una vez que se copian, se realiza el scroll de la página. Se espera un tiempo definido en la función de scrolling para que cargue antes de seguir buscando.

se recomienda iniciar el browser por python **1)**, luego ir al buscador avanzado de twitter(https://twitter.com/search-advanced), ingresar los términos de busqueda deseados, ir a latest y ejecutar esta rutina. paso **2)** no es necesario.

In [ ]:
# rutina de extracción de tweets
while not end_of_scroll_region:
    cards = collect_all_tweets_from_current_view(driver)
    for card in cards:
        try:
            tweet = extract_data_from_current_tweet_card(card)
        except exceptions.StaleElementReferenceException:
            continue
        if not tweet:
            continue
        tweet_id = generate_tweet_id(tweet)
        if tweet_id not in unique_tweets:
            unique_tweets.add(tweet_id)
            save_tweet_data_to_csv(tweet, filepath)
    last_position, end_of_scroll_region = scroll_down_page(driver, last_position)

**Lectura de información:** En esta sección se lee el archivo .csv que se produjo.

Este archivo corresponde al resultado de copia de todos los tweets buscados según los filtros: #santanderchile #bancochile #estasestamos #bancoestado #bancobci y bancodechile bancochile bancosantander santanderchile bancobci bsantander bancoedwards bedwards banchile

Para esto, se inició el browser desde Python con el paso **1)** y posteriormente se fue a la búsqueda avanzada manualmente y se ingresaron los términos de búsqueda desde 2006 hasta 2021. Luego se corrió la rutina de extracción **3)**


In [3]:
# se lee el archivo .csv en este caso santander_chile_estado_bci.csv
datos = pd.read_csv(os.path.join("santander_chile_estado_bci.csv"))     
datos.head() 

,User,Handle,PostDate,TweetText,ReplyCount,RetweetCount,LikeCount
0,Ceci (Winthertur),@Ceci83437135,2021-01-18T15:00:34.000Z,"Excelente, no lo sabia, pero no lo publicitan ...",NaN,NaN,NaN
1,Leonardo,@tijercat,2021-01-18T13:24:03.000Z,"Sres \n@BancoEstado\n como no tener ""Tino"", al...",NaN,NaN,NaN
2,“ Carola Correa“ #YoVotoRechazo,@carolincorreav,2021-01-15T15:54:49.000Z,"Replying to \n@BancoEstadoChao #bancoestado, n...",NaN,NaN,NaN
3,HCN,@DelCantoNunez,2021-01-13T22:34:42.000Z,"Muy cierto, algunos compatriotas dejan fluir s...",NaN,NaN,NaN
4,Urián Cádiz,@urian_cadiz,2021-01-12T18:03:06.000Z,Utilidad pública.... Ninguna institución banca...,NaN,NaN,1.0


**Filtro de clasificación:** se crean columnas con los nombres de los bancos buscados. En estas columnas se escribe -1 si no se menciona el banco y 1 si se mencionaa.

In [5]:
df = datos.copy() # copia de datos

In [6]:
# se crean nuevas columnas en el DataFrame para clasificación
df["BancoEstado"]= 0
df["BancoChile"]= 0
df["Santander"]= 0
df["BCI"]= 0

In [7]:
df["TweetText"] = df["TweetText"].str.lower() # se cambian todas las mayúsculas a minúsculas para no diferenciar e.g. Banco con banco

**Clasificación:** el método .find() retorna un número. -1 si no se encuentra la palabra buscada en el string y un número de posición (positivo) si se encuentra.

In [8]:
df["BancoEstado"] = df["TweetText"].str.find("bancoestado")
df["Santander"] = df["TweetText"].str.find("santander") # b
df["BancoChile"] = df["TweetText"].str.find("bancochile")
df["BCI"] = df["TweetText"].str.find("bci")

In [41]:
df

,User,Handle,PostDate,TweetText,ReplyCount,RetweetCount,LikeCount,BancoEstado,BancoChile,Santander,BCI
0,Ceci (Winthertur),@Ceci83437135,2021-01-18T15:00:34.000Z,"excelente, no lo sabia, pero no lo publicitan ...",NaN,NaN,NaN,-1,59,-1,53
1,Leonardo,@tijercat,2021-01-18T13:24:03.000Z,"sres \n@bancoestado\n como no tener ""tino"", al...",NaN,NaN,NaN,7,-1,152,-1
2,“ Carola Correa“ #YoVotoRechazo,@carolincorreav,2021-01-15T15:54:49.000Z,"replying to \n@bancoestadochao #bancoestado, n...",NaN,NaN,NaN,14,-1,-1,-1
3,HCN,@DelCantoNunez,2021-01-13T22:34:42.000Z,"muy cierto, algunos compatriotas dejan fluir s...",NaN,NaN,NaN,-1,155,-1,-1
4,Urián Cádiz,@urian_cadiz,2021-01-12T18:03:06.000Z,utilidad pública.... ninguna institución banca...,NaN,NaN,1.0,168,-1,157,-1
...,...,...,...,...,...,...,...,...,...,...,...
6332,Max y Miliano Martin,@maxymiliano,2009-11-03T18:11:57.000Z,pagando un nic ... el webpay no me llevó al ba...,NaN,NaN,NaN,-1,119,-1,-1
6333,Ale Silberstein,@alesilberstein,2009-10-31T17:00:42.000Z,rt \n@juanpablotapia\n: sabía usté que el #ban...,NaN,NaN,NaN,-1,41,-1,-1
6334,nodalomismo,@paulisalazar,2009-10-26T15:57:41.000Z,nada más enredado que el #bancochile para abri...,NaN,NaN,NaN,-1,26,-1,-1
6335,Neotronics SPA,@neotronicscl,2009-10-13T19:51:58.000Z,peleando con #bancochile.... arghhh!,NaN,NaN,NaN,-1,14,-1,-1


In [9]:
df["BancoEstado"][ df["BancoEstado"] > -1] = 1 
df["Santander"][ df["Santander"] > -1] = 1 
df["BancoChile"][ df["BancoChile"] > -1] = 1 
df["BCI"][ df["BCI"] > -1] = 1 

<ipython-input-9-307c0ceab47a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["BancoEstado"][ df["BancoEstado"] > -1] = 1
<ipython-input-9-307c0ceab47a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Santander"][ df["Santander"] > -1] = 1
<ipython-input-9-307c0ceab47a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["BancoChile"][ df["BancoChile"] > -1] = 1
<ipython-input-9-307c0ceab47a>:4: SettingWithCopyWarning: 
A val

In [10]:
df = df.fillna(0)

In [11]:
df2 = df.drop(['User','Handle'], inplace=False, axis=1)
df2

,PostDate,TweetText,ReplyCount,RetweetCount,LikeCount,BancoEstado,BancoChile,Santander,BCI
0,2021-01-18T15:00:34.000Z,"excelente, no lo sabia, pero no lo publicitan ...",0.0,0.0,0.0,-1,1,-1,1
1,2021-01-18T13:24:03.000Z,"sres \n@bancoestado\n como no tener ""tino"", al...",0.0,0.0,0.0,1,-1,1,-1
2,2021-01-15T15:54:49.000Z,"replying to \n@bancoestadochao #bancoestado, n...",0.0,0.0,0.0,1,-1,-1,-1
3,2021-01-13T22:34:42.000Z,"muy cierto, algunos compatriotas dejan fluir s...",0.0,0.0,0.0,-1,1,-1,-1
4,2021-01-12T18:03:06.000Z,utilidad pública.... ninguna institución banca...,0.0,0.0,1.0,1,-1,1,-1
...,...,...,...,...,...,...,...,...,...
6332,2009-11-03T18:11:57.000Z,pagando un nic ... el webpay no me llevó al ba...,0.0,0.0,0.0,-1,1,-1,-1
6333,2009-10-31T17:00:42.000Z,rt \n@juanpablotapia\n: sabía usté que el #ban...,0.0,0.0,0.0,-1,1,-1,-1
6334,2009-10-26T15:57:41.000Z,nada más enredado que el #bancochile para abri...,0.0,0.0,0.0,-1,1,-1,-1
6335,2009-10-13T19:51:58.000Z,peleando con #bancochile.... arghhh!,0.0,0.0,0.0,-1,1,-1,-1


In [12]:
df2["PostDate"] = df2["PostDate"].apply(lambda tweet: tweet.split("T")[0])

In [13]:
df2

,PostDate,TweetText,ReplyCount,RetweetCount,LikeCount,BancoEstado,BancoChile,Santander,BCI
0,2021-01-18,"excelente, no lo sabia, pero no lo publicitan ...",0.0,0.0,0.0,-1,1,-1,1
1,2021-01-18,"sres \n@bancoestado\n como no tener ""tino"", al...",0.0,0.0,0.0,1,-1,1,-1
2,2021-01-15,"replying to \n@bancoestadochao #bancoestado, n...",0.0,0.0,0.0,1,-1,-1,-1
3,2021-01-13,"muy cierto, algunos compatriotas dejan fluir s...",0.0,0.0,0.0,-1,1,-1,-1
4,2021-01-12,utilidad pública.... ninguna institución banca...,0.0,0.0,1.0,1,-1,1,-1
...,...,...,...,...,...,...,...,...,...
6332,2009-11-03,pagando un nic ... el webpay no me llevó al ba...,0.0,0.0,0.0,-1,1,-1,-1
6333,2009-10-31,rt \n@juanpablotapia\n: sabía usté que el #ban...,0.0,0.0,0.0,-1,1,-1,-1
6334,2009-10-26,nada más enredado que el #bancochile para abri...,0.0,0.0,0.0,-1,1,-1,-1
6335,2009-10-13,peleando con #bancochile.... arghhh!,0.0,0.0,0.0,-1,1,-1,-1


In [17]:
df2["PostDate"][3]

'2021-01-13'

In [78]:
df2[df2["TweetText"].str.find("replying to") >-1 ]
df2["TweetText"].str.find("replying to") >-1

0        True
1       False
2        True
3       False
4       False
        ...  
6332    False
6333    False
6334    False
6335    False
6336    False
Name: TweetText, Length: 6337, dtype: bool

In [94]:
#df2.loc[:,("TweetText")]
df2.loc[df2["TweetText"].str.find("replying to") == 0, 'TweetText']
#f2["TweetText"][2].find("replying to")

2       replying to \n@bancoestadochao #bancoestado, n...
7       replying to \n@renatogaring\n and \n@garindipu...
23      replying to \n@chilezuela1973llevo 19 dias esp...
26      replying to \n@elquenoaportallevo 18 dias espe...
27      replying to \n@paaulimarurillevo 18 dias esper...
                              ...                        
6228    replying to \n@arttheco@coespi asaltaron el #b...
6240    replying to \n@bancodechile@bancodechile\n asi...
6256    replying to \n@val_alvarezf@v4l89 #loaton? ¿qu...
6325    replying to \n@rodboc@rodcl\n woohoo..al fin v...
6327    replying to \n@jorgebarahona@jorgebarahona\n n...
Name: TweetText, Length: 578, dtype: object

In [99]:
print(df2["TweetText"][6228])

replying to 
@arttheco@coespi asaltaron el #bancochile que está en suecia con 11 de septiembre
